## Setup

In [1]:
import os 

is_kaggle = False
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None:
    is_kaggle = True

In [2]:
%%capture
if is_kaggle:
    !pip install ../input/sklearn-1-0/scikit_learn-1.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

In [3]:
from pathlib import Path
from typing import Tuple, List

import pandas as pd
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [4]:
seed = 32

In [5]:
is_kaggle = False
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None:
    is_kaggle = True

In [6]:
data_path = Path("..") / "data" / "raw"
output_path = Path("..") / 'data' / "submissions"
output_file_name = 'lg.csv'
if is_kaggle:
    data_path = Path('/kaggle') / 'input' / 'jigsaw-unintended-bias-in-toxicity-classification'
    output_path = Path("/kaggle") / "working"
    output_file_name = 'submission.csv'

In [7]:
data_path

PosixPath('../data/raw')

In [8]:
identity_columns = [
    "male",
    "female",
    "homosexual_gay_or_lesbian",
    "christian",
    "jewish",
    "muslim",
    "black",
    "white",
    "psychiatric_or_mental_illness",
]

## Load Data

In [9]:
df = pd.read_csv(data_path / 'train.csv')

In [10]:
df.head(2)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.0,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.0,Thank you!! This would make my life a lot less...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4


In [11]:
df['label'] = (df['target'] >= 0.5).astype(int)

In [12]:
x = df['comment_text']
y = df['label']

In [13]:
def convert_dataframe_to_bool(
    df: pd.DataFrame, identity_columns: List[str] = identity_columns
) -> pd.DataFrame:
    """Convert identity columns to boolen columns"""
    bool_df = df.copy()
    for col in identity_columns:
        bool_df[col] = np.where(df[col] >= 0.5, True, False)
    return bool_df


df = convert_dataframe_to_bool(df)


In [14]:
def get_x_y(df: pd.DataFrame, input_col='comment_text', label_col='label') -> Tuple[pd.DataFrame, pd.DataFrame]:
    return df[input_col], df[label_col]

In [15]:
df_train, df_valid = train_test_split(df, test_size=0.2, random_state=seed)

In [16]:
x_train, y_train= get_x_y(df_train)
x_valid, y_valid = get_x_y(df_valid)

## Preprocess Data

In [17]:
train_text = list(x_train.values)
valid_text = list(x_valid.values)

In [18]:
vectorizer = CountVectorizer(
    stop_words="english", max_features=5000, min_df=0.001, max_df=0.99
)
x_train_prepared = vectorizer.fit_transform(train_text)
x_valid_prepared = vectorizer.transform(valid_text)


In [19]:
x_train_prepared.shape, x_valid_prepared.shape

((1443899, 3691), (360975, 3691))

In [20]:
vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'zero', 'zone', 'zuma'], dtype=object)

In [21]:
joblib.dump(vectorizer, "../models/bow/vectorizer.joblib")

['../models/bow/vectorizer.joblib']

## Train Model

In [22]:
mod_lg = LogisticRegression(random_state=seed, solver='liblinear')

In [23]:
_ = mod_lg.fit(x_train_prepared, y_train)

In [24]:
joblib.dump(mod_lg, "../models/bow/logistic.joblib")

['../models/bow/logistic.joblib']

## Evaluate Model

In [25]:
def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan


def compute_subgroup_auc(
    df: pd.DataFrame, subgroup: str, label_col: str, y_proba_col: str
) -> float:
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label_col], subgroup_examples[y_proba_col])


def compute_bpsn_auc(df, subgroup, label, y_proba_col):
    """
    Computes the AUC of the within-subgroup negative examples
    and the background positive examples.
    """
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[y_proba_col])


def compute_bnsp_auc(df, subgroup, label, y_proba_col):
    """
    Computes the AUC of the within-subgroup positive examples and the
    background negative examples.
    """
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[y_proba_col])


def compute_bias_metrics_for_model(
    dataset: pd.DataFrame, subgroups: List[str], y_proba_col: str, label_col: str
):
    """Computes per-subgroup metrics for all subgroups"""
    records = []
    for subgroup in subgroups:
        record = {
            "subgroup": subgroup,
            "subgroup_size": len(dataset[dataset[subgroup]]),
        }
        record["subgroup_auc"] = compute_subgroup_auc(
            dataset, subgroup, label_col, y_proba_col
        )
        # background positive, subgroup negative
        record["bpsn_auc"] = compute_bpsn_auc(dataset, subgroup, label_col, y_proba_col)
        # background negative, subgroup positive
        record["bnsp_auc"] = compute_bnsp_auc(dataset, subgroup, label_col, y_proba_col)
        records.append(record)
    return pd.DataFrame(records).sort_values("subgroup_auc", ascending=True)


In [26]:
def get_freq_table(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """Get the count and percentage of each unique value in the column"""
    num_count = df[col].value_counts()
    perc_count = df[col].value_counts(normalize=True)
    df_sum = pd.concat([num_count, perc_count], axis=1)
    df_sum.columns = ["count", "percentage"]
    return df_sum

In [27]:
def evaluate_model(df: pd.DataFrame, label_col: str = "label") -> pd.DataFrame:
    y_true = df[label_col].values
    y_pred = df["y_pred"].values
    y_proba = df["y_pred_proba"].values

    acc = metrics.accuracy_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    auc_roc = metrics.roc_auc_score(y_true, y_proba)

    df_result = pd.DataFrame(
        {"metrics": ["accuracy", "f1", "auc_roc"], "value": [acc, f1, auc_roc]}
    )
    return df_result


In [28]:
df_eval = df_valid.reset_index(drop=True).copy()

In [29]:
df_eval['y_pred'] = mod_lg.predict(x_valid_prepared)
df_eval['y_pred_proba'] = mod_lg.predict_proba(x_valid_prepared)[:, 1]

In [30]:
df_eval.head(2)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,label,y_pred,y_pred_proba
0,310214,0.0,I am not suggesting that the sate run an oil c...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,0,0,0,0,0.0,0,4,0,0,0.005680
1,344184,0.0,"Aloha Luke, another great column. We have bee...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,0,0,1,0,0.0,0,4,0,0,0.002334


In [31]:
get_freq_table(df_valid, col='label')

,count,percentage
0,332132,0.920097
1,28843,0.079903


In [32]:
bias_metrics_df = compute_bias_metrics_for_model(df_eval, identity_columns, 'y_pred_proba', 'label')
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
2,homosexual_gay_or_lesbian,2208,0.748368,0.763592,0.876749
6,black,3107,0.752519,0.708626,0.921839
7,white,5016,0.775468,0.717442,0.929837
5,muslim,4206,0.784206,0.750862,0.921171
4,jewish,1513,0.806274,0.803642,0.888003
8,psychiatric_or_mental_illness,991,0.816129,0.780970,0.923800
0,male,8926,0.838875,0.820773,0.896875
1,female,10774,0.841832,0.837805,0.886515
3,christian,8341,0.862200,0.885028,0.857405


In [33]:
evaluate_model(df_eval)

,metrics,value
0,accuracy,0.939295
1,f1,0.491920
2,auc_roc,0.881184


## Explain Model

In [34]:
df_coef = pd.DataFrame({"name": vectorizer.get_feature_names_out(), "coef": mod_lg.coef_[0]})
df_coef.sort_values('coef', ignore_index=True)

,name,coef
0,knee,-1.296578
1,april,-0.765470
2,rare,-0.697855
3,discussions,-0.690627
4,steps,-0.658778
...,...,...
3686,moron,4.526702
3687,stupid,4.832190
3688,stupidity,4.934228
3689,idiots,5.461493


## Make Submission

In [35]:
df_test = pd.read_csv(data_path / 'test.csv')

In [36]:
df_test.head(2)

,id,comment_text
0,7097320,[ Integrity means that you pay your debts.]\n\...
1,7097321,This is malfeasance by the Administrator and t...


In [37]:
x_test_prepared = vectorizer.transform(list(df_test.comment_text.values))

In [38]:
prediction = mod_lg.predict_proba(x_test_prepared)[:, 1]

In [39]:
prediction

array([0.05108628, 0.06198041, 0.03860329, ..., 0.05951157, 0.16083905,
       0.00889832])

In [40]:
df_submit = pd.DataFrame({'id': df_test.id.values, 'prediction': prediction})
df_submit.to_csv(output_path / output_file_name, index=False)

## Calculate Submission Metrics

In [41]:
def calculate_overall_auc(df, model_name):
    true_labels = df["label"]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)


def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)


def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average(
        [
            power_mean(bias_df["subgroup_auc"], POWER),
            power_mean(bias_df["bpsn_auc"], POWER),
            power_mean(bias_df["bnsp_auc"], POWER),
        ]
    )
    return (OVERALL_MODEL_WEIGHT * overall_auc) + (
        (1 - OVERALL_MODEL_WEIGHT) * bias_score
    )


In [42]:
df_test_complete  = pd.read_csv(data_path / 'test_private_expanded.csv')
df_test_complete = convert_dataframe_to_bool(df_test_complete)

In [43]:
df_test_complete['y_pred_proba'] = prediction
df_test_complete['y_pred'] = (prediction >= 0.5).astype(int)
df_test_complete['label'] = (df_test_complete.toxicity >= 0.5).astype(int)

In [44]:
evaluate_model(df_test_complete)

,metrics,value
0,accuracy,0.938440
1,f1,0.483312
2,auc_roc,0.879092


In [45]:
bias_metrics_df = compute_bias_metrics_for_model(df_test_complete, identity_columns, 'y_pred_proba', 'label')

In [46]:
get_final_metric(bias_metrics_df, calculate_overall_auc(df_test_complete, 'y_pred_proba'))

0.8322929932994851